Redox flow battery setup using `rfbmodel`

In [1]:
# Change for your directory 
# Remember to pip install . after each change from the
# rfbmodel directory
#!pip install /home/felipe/RFBmodel/ 
!pip install "C:/Users/jbvba/RFBmodelPC/RFBmodel"


Processing c:\users\jbvba\rfbmodelpc\rfbmodel
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rfbmodel: filename=rfbmodel-0.1-py3-none-any.whl size=6493 sha256=6c4674486635039db23548d3e33a842bbd9105a5f358260ed9c8b50b2e0aed63
  Stored in directory: C:\Users\jbvba\AppData\Local\Temp\pip-ephem-wheel-cache-czs24qyd\wheels\f5\e2\e6\adaf62f013c25d2ed8d5494892f41e9f9fab1d4b2646ed6396
Successfully built rfbmodel
  Attempting uninstall: rfbmodel
    Found existing installation: rfbmodel 0.1
    Uninstalling rfbmodel-0.1:
      Successfully uninstalled rfbmodel-0.1


  DEPRECATION: Building 'rfbmodel' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rfbmodel'. Discussion can be found at https://github.com/pypa/pip/issues/6334


It is always advisable to check the code in a script before converting it into a class. For example, we will test the `conc` function from the cathode.

Suggestion: use SciPy to solve the non linear system

$$ f(\vec{c}) = \vec{b}$$

Now this is implemented in the cathode class

In [2]:
from rfbmodel.cathodes import Cathode
from rfbmodel.cells import Cell
from rfbmodel.agdl import AGDL
from rfbmodel.acl import ACL
from rfbmodel.data import conductivity_funcV5, conductivity_funcV4

In [3]:
#Define design parameters for cathode

cath_thickness = 50/1000 #Cathode thickness [m]
cath_conductivity = 4400 #Conductivity of the cathode / sigma [S m^-1]
cath_porosity = 0.94 #Cathode porosity (void fraction)

# Construct a cathode
cath = Cathode(cath_thickness, cath_conductivity, cath_porosity)

# Print initial guess
print("c_guess = ", cath.c)

# Solve system of equations
cath.conc()

# Print solution
print("c = ", cath.c)



c_guess =  [1.   1.   1.   1.   2.   0.01]
c =  [ 1.03000000e+00 -1.07292877e-10  1.02985342e+01  8.42961216e+00
  1.96446103e+00  4.35151124e-01]


In [4]:
cath.ionic_resistance()
cath.cathode_resistance()
cath.total_resistance()

# Print solution
print("resistance of cathode = ", cath.cath_resistance, "Ωm2")
print("resistance of catholyte = ", cath.io_resistance, "Ωm2")
print("Total resistance of cathode and catholyte = ", cath.resistance, "Ωm2")


resistance of cathode =  0.0007731975198179214 Ωm2
resistance of catholyte =  0.0017908546514708034 Ωm2
Total resistance of cathode and catholyte =  0.000540037519663905 Ωm2


AGDL Class

In [ ]:
# Define design parameters for AGDL
agdl_thickness = 171*10**(-6) #AGDL thickness [m] 
agdl_conductivity = 8700 # AGDL Conductivity[S m^−1]
agdl_porosity = 0.6 #AGDL porosity (void fraction)

# Construct a Anode Gas Diffusion Layer
agdl = AGDL(agdl_thickness,agdl_conductivity,agdl_porosity)

# Calculate AGDL resistance
agdl.calculate_resistance()

# Print solution
print("AGDL Resistance", agdl.resistance, "Ωm2")

AGDL Resistance 4.972409010471659e-09 Ωm2


ACL Class

In [ ]:
# Define design parameters for ACL
acl_thickness = 19/10**6 #Anode catalyst layer thickness [m]
acl_conductivity= 240 #ACL electronic conductivity [S m^-1]
acl_porosity = 0.4 #Anode catalyst layer porosity

# Construct a Anode Catalyst Layer
acl = ACL(acl_thickness, acl_conductivity, acl_porosity)

# Calculate ACL resistance
acl.calculate_resistance()

# Print solution
print("ACL Resistance", agdl.resistance, "Ωm2")

ACL Resistance 4.972409010471659e-09 Ωm2


Cell Class

In [ ]:
#Construct a plain dictionary for Standard Cell Potential and Standard Entropy of the reaction
reaction_props1 = {
    'E_SCP': 0.99,     # Standard Cell Potential of reaction [V]
    'delta_S_r': -86.9   # Standard Entropy of the cell discharge reaction [J/mol/K]
}

#Define operation parameters for the cell
T1 = 305.15 #Cell temperature [K]
SOC1 = 0.5 #State of charge {0,1}
Q_v1 = 45 #Catholyte flow rate [mL/min]
j_appl1 = 1000 #Current density [A/m^2]

# Construct a Cell 
### `Cell(cathode, membrane, acl, agdl, T, SOC, Q_v, j_appl, reaction_props)`
cell=Cell(cathode = cath , membrane = None, acl = acl, agdl = agdl, T = T1, SOC = SOC1, Q_v = Q_v1, j_appl = j_appl1, reaction_props = reaction_props1)

#Method for the formal cell potential, dependent on temperature
E0_cell=cell.E0_cell()
print("Formal Cell Potential", E0_cell, "V at", cell.T, "Kelvin")

#Calculates total cell voltage based on: Open circuit potential, Membrane ohmic loss, Cathode overpotential, 
# Anode overpotential, Protonic transport in ACL - NOT READY YET
#E_cell=cell.E_cell()
#print("Cell Potential", E_cell, "V, on", cell.T, "Kelvin")

#Method for the high frequency resistance, which accounts for the electronic and ionic resistance of the cathode, catholyte, ACL, AGDL, 
# membrane, current collector and bipolar plates
R_HFR=cell.R_HFR()
print("HFR is", R_HFR, "omega m^2")


Formal Cell Potential 0.9868476965331399 V at 305.15 Kelvin
HFR is 0.00048148596397645 omega m^2
